In [ ]:
# ==============================================================================
# PROJECT: THELOOK ECOMMERCE - ADVANCED DATA ANALYTICS & BI
# Author: Christian Panazzolo
# Objective: Data extraction, statistical processing, and interactive visualization
# ==============================================================================

# --- 1. Dependencies and Environment Setup ---
# Importing core libraries for data manipulation and high-level visualization
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Authentication for Google Cloud Services
auth.authenticate_user()
print('GCP Authentication Successful')

# Initialize BigQuery Client
project_id = 'vizportfolio'
client = bigquery.Client(project=project_id)

# --- 2. Data Extraction Strategy ---
# SQL Query designed to join Orders, Items, and Products for a holistic view.
# Filtering out 'Cancelled' orders to ensure financial accuracy.
query_base = """
SELECT
    o.order_id,
    o.user_id,
    o.status,
    o.created_at,
    oi.sale_price,
    p.category,
    p.name AS product_name,
    p.retail_price,
    p.cost,
    (oi.sale_price - p.cost) AS profit
FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
    ON o.order_id = oi.order_id
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
    ON oi.product_id = p.id
WHERE o.status != 'Cancelled'
"""

# Converting BigQuery result set to a Pandas DataFrame
df = client.query(query_base).to_dataframe()

# Inspecting dataset structure and schema
print(f"Dataset Loaded: {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()

# --- 3. Advanced Visualization Engineering ---
# Note: The following cells generate the D3.js interactive components
# that are embedded into the Looker Studio dashboard via GitHub Pages.


GCP Authentication Successful
Dataset Loaded: 154089 rows and 10 columns.


,order_id,user_id,status,created_at,sale_price,category,product_name,retail_price,cost,profit
0,1090,901,Complete,2024-12-02 18:26:27+00:00,62.990002,Clothing Sets,BLACK BLOUSE GAUCHO PALAZZO SET ASYM - FITS - ...,62.990002,38.297921,24.692081
1,5323,4252,Complete,2025-09-28 20:47:37+00:00,99.989998,Clothing Sets,Woman Within Plus Size Petite rhinestone-trim ...,99.989998,66.993298,32.996699
2,5938,4747,Complete,2024-11-30 09:25:09+00:00,62.990002,Clothing Sets,GREY BLOUSE GAUCHO PALAZZO SET ASYM - FITS - 3...,62.990002,37.919981,25.070021
3,7150,5720,Complete,2025-04-22 21:39:57+00:00,89.989998,Clothing Sets,Sutton Studio Womens Black Leopard Silk Blend ...,89.989998,52.104208,37.885789
4,9538,7656,Complete,2024-07-19 01:30:36+00:00,62.990002,Clothing Sets,BLACK BLOUSE GAUCHO PALAZZO SET ASYM - FITS - ...,62.990002,38.297921,24.692081


In [ ]:
# --- BUBBLE CHART OF MARGIN AND REVENUE GROUPED BY CATEGORIES ---

# Calculate required metrics per category
category_metrics = df.groupby('category').agg(
    avg_margin_per_unit=('profit', 'mean'),
    revenue=('sale_price', 'sum'),
    total_profit=('profit', 'sum')
).reset_index()

# Sort by total_profit and select the top 10 categories
category_metrics_top10 = category_metrics.sort_values(by='total_profit', ascending=False).head(10)

# Creating an interactive bubble chart
fig = px.scatter(
    category_metrics_top10,
    x="category",
    y="avg_margin_per_unit",
    size="revenue",
    color="total_profit",
    hover_name="category",
    title="Product Mix Analysis: Top 10 Categories by Profit",
    labels={
        "category": "Product Category",
        "avg_margin_per_unit": "Avg Margin per Unit ($)",
        "revenue": "Total Revenue",
        "total_profit": "Total Profit ($)"
    },
    color_continuous_scale=px.colors.sequential.Blues
)

# Refining layout
fig.update_layout(
    template="plotly_white",
    font=dict(family="Arial, sans-serif", size=12),
    margin=dict(l=50, r=50, t=80, b=50),
    title_font_size=24,
    coloraxis_colorbar=dict(title="Profit ($)")
)

# Customizing the marker border for better visibility
fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))

fig.show()

# Exporting into html
file_name = "advanced_analysis_plotly_en.html"
fig.write_html(file_name)
print(f"Success! {file_name} has been generated.")

Success! advanced_analysis_plotly_en.html has been generated.


In [ ]:
# --- HEATMAP: ORDERS BY DAY AND HOUR ---
query_heatmap = """
SELECT
    EXTRACT(DAYOFWEEK FROM created_at) AS day_of_week,
    EXTRACT(HOUR FROM created_at) AS hour_of_day,
    COUNT(*) AS total_orders
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at >= '2023-01-01'
GROUP BY 1, 2
"""
df_heatmap = client.query(query_heatmap).to_dataframe()

# Pivot data for heatmap format
heatmap_data = df_heatmap.pivot(index='day_of_week', columns='hour_of_day', values='total_orders')

fig_heatmap = px.imshow(
    heatmap_data,
    labels=dict(x="Hour of Day", y="Day of Week (1=Sun)", color="Orders"),
    x=list(range(24)),
    y=['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'],
    title="Order Density: Sales Peak Analysis",
    color_continuous_scale='YlGnBu'
)

fig_heatmap.update_layout(template="plotly_white")
fig_heatmap.show()

# Export
fig_heatmap.write_html("order_density_heatmap.html")

In [ ]:
# --- BOX PLOT: PRICE DISTRIBUTION & OUTLIERS (ORDERED) ---
query_box = """
SELECT
    p.category,
    oi.sale_price
FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
JOIN `bigquery-public-data.thelook_ecommerce.products` AS p ON oi.product_id = p.id
WHERE p.category IN ('Outerwear & Coats', 'Sweaters', 'Jeans', 'Suits & Sport Coats')
"""
df_box = client.query(query_box).to_dataframe()

import plotly.express as px

# Defining order of columns to match the previous graph
target_order = ['Outerwear & Coats', 'Sweaters', 'Jeans', 'Suits & Sport Coats']

fig_box = px.box(
    df_box,
    x="category",
    y="sale_price",
    color="category",
    category_orders={"category": target_order},
    title="Price Dispersion & Outlier Detection by Category",
    labels={"category": "Category", "sale_price": "Sale Price ($)"},
    points="outliers"
)

fig_box.update_layout(
    template="plotly_white",
    showlegend=False,
    xaxis_title="Product Category",
    yaxis_title="Sale Price ($)"
)

fig_box.show()

# Export
fig_box.write_html("price_distribution_boxplot.html")

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Authentication ---
auth.authenticate_user()
# Reuse the project_id defined earlier in the notebook
client = bigquery.Client(project=project_id)

# --- 2. Data Extraction ---
# Querying orders over time to calculate half-yearly metrics
query_time_series = """
SELECT
    oi.created_at,  -- Fetch raw timestamp
    u.id as user_id,
    oi.sale_price
FROM `bigquery-public-data.thelook_ecommerce.users` AS u
JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi ON u.id = oi.user_id
WHERE u.country = 'United States'
"""

df = client.query(query_time_series).to_dataframe()


)

# Calculate the half-year (1H for Jan-Jun, 2H for Jul-Dec)
df['created_at'] = pd.to_datetime(df['created_at']
df['year'] = df['created_at'].dt.year
df['half'] = ((df['created_at'].dt.month - 1) // 6) + 1
df['half_year'] = df['year'].astype(str) + '-' + df['half'].astype(str) + 'H'

half_yearly_data = df.groupby('half_year').agg(
    unique_buyers=('user_id', 'nunique'),
    avg_spend=('sale_price', 'mean')
).reset_index()

# Sort the data by half_year to ensure correct chronological order on the chart
half_yearly_data = half_yearly_data.sort_values(by='half_year')

# --- 4. Visualization with Dual Y-Axis ---
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add Bar Chart for Unique Buyers
fig.add_trace(
    go.Bar(
        x=half_yearly_data['half_year'],
        y=half_yearly_data['unique_buyers'],
        name="Unique Buyers",
        marker_color='#1f77b4',
        opacity=0.7
    ),
    secondary_y=False,
)

# Add Line Chart for Average Spend
fig.add_trace(
    go.Scatter(
        x=half_yearly_data['half_year'],
        y=half_yearly_data['avg_spend'],
        name="Avg Spend ($)",
        mode='lines+markers',
        line=dict(color='#ff7f0e', width=3)
    ),
    secondary_y=True,
)

# --- 5. Layout Styling ---
fig.update_layout(
    title_text="Half-Yearly Growth: Unique Buyers vs. Average Spend",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    autosize=True,
    margin=dict(l=0, r=0, t=0, b=0),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Setting axis titles and removing gridlines
fig.update_yaxes(title_text="Number of Buyers", secondary_y=False, showgrid=False)
fig.update_yaxes(title_text="Average Spend ($)", secondary_y=True, showgrid=False)
fig.update_xaxes(showgrid=False)

fig.show()

# Export
fig.write_html("growth_comparison_dual_axis.html")

In [ ]:
# Aggregating global state metrics and identifying the most popular category
query_map_data = f"""
WITH state_general_metrics AS (
    SELECT
        u.state,
        SUM(oi.sale_price) as total_revenue,
        COUNT(DISTINCT oi.order_id) as total_orders,
        COUNT(DISTINCT u.id) as unique_buyers
    FROM `bigquery-public-data.thelook_ecommerce.users` AS u
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi ON u.id = oi.user_id
    WHERE u.country = 'United States'
    GROUP BY 1
),
category_ranking AS (
    SELECT
        u.state,
        p.category,
        COUNT(oi.id) as category_count,
        RANK() OVER (PARTITION BY u.state ORDER BY COUNT(oi.id) DESC) as rank_cat
    FROM `bigquery-public-data.thelook_ecommerce.users` AS u
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi ON u.id = oi.user_id
    JOIN `bigquery-public-data.thelook_ecommerce.products` AS p ON oi.product_id = p.id
    WHERE u.country = 'United States'
    GROUP BY 1, 2
)
SELECT
    g.state,
    g.total_revenue,
    g.total_orders,
    g.unique_buyers,
    c.category as top_category
FROM state_general_metrics g
JOIN category_ranking c ON g.state = c.state
WHERE c.rank_cat = 1
"""

df_map = client.query(query_map_data, project=project_id).to_dataframe()

# --- Mapping state names to codes ---
df_map['state_code'] = df_map['state'].str.strip().map(us_state_to_abbrev)

# --- Color scale adjustment (90th percentile for better contrast) ---
color_max = df_map['total_revenue'].quantile(0.9)

# --- Choropleth Map configuration ---
fig_map = px.choropleth(
    df_map,
    locations='state_code',
    locationmode="USA-states",
    color='total_revenue',
    hover_name='state',
    hover_data={
        'state_code': False,
        'total_revenue': ':$.2f',
        'total_orders': True,
        'unique_buyers': True,
        'top_category': True
    },
    color_continuous_scale="Blues",
    range_color=[0, color_max],
    scope="usa",
    labels={
        'total_revenue': 'Total Revenue ($)',
        'total_orders': 'Orders',
        'unique_buyers': 'Unique Buyers',
        'top_category': 'Top Category'
    }
)

# --- Layout styling for Looker Studio integration ---
fig_map.update_layout(
    geo=dict(bgcolor='rgba(0,0,0,0)'),
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=0, r=0, t=0, b=0),
    autosize=True
)

# --- Render and export ---
fig_map.show()
fig_map.write_html("us_market_map_complete_analysis.html", include_plotlyjs='cdn', full_html=False)

In [ ]:
# --- Adjusted Behavioral Analysis: Top 5 States ---
# Normalized by Maximum instead of Min-Max to avoid zeroing out the lowest values
query_radar_fixed = f"""
WITH top_5_states AS (
    SELECT
        u.state,
        SUM(oi.sale_price) as state_revenue
    FROM `bigquery-public-data.thelook_ecommerce.users` AS u
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi ON u.id = oi.user_id
    WHERE u.country = 'United States'
    GROUP BY 1
    ORDER BY state_revenue DESC
    LIMIT 5
),
state_behavior AS (
    SELECT
        u.state,
        SUM(oi.sale_price) as total_revenue,
        AVG(oi.sale_price) as avg_ticket,
        AVG(user_cat_count) as category_diversity,
        CAST(COUNT(oi.id) / COUNT(DISTINCT u.id) AS FLOAT64) as purchase_frequency
    FROM (
        SELECT u.id, u.state, COUNT(DISTINCT p.category) as user_cat_count
        FROM `bigquery-public-data.thelook_ecommerce.users` u
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` oi ON u.id = oi.user_id
        JOIN `bigquery-public-data.thelook_ecommerce.products` p ON oi.product_id = p.id
        GROUP BY 1, 2
    ) u
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` oi ON u.id = oi.user_id
    WHERE u.state IN (SELECT state FROM top_5_states)
    GROUP BY 1
)
SELECT * FROM state_behavior
"""

df_radar = client.query(query_radar_fixed, project=project_id).to_dataframe()

# --- Normalization Logic (Max-Based) ---
# Each value becomes a % of the maximum value of that metric
df_norm = df_radar.copy()
axis_metrics = ['total_revenue', 'avg_ticket', 'category_diversity', 'purchase_frequency']

for m in axis_metrics:
    max_val = df_norm[m].max()
    if max_val == 0:
        df_norm[m] = 0
    else:
        df_norm[m] = df_norm[m] / max_val

# --- Prepare data for Plotly Express ---
df_melted = df_norm.melt(id_vars='state', var_name='metric', value_name='value')

# --- Final Radar Chart with distinct state colors ---
fig_radar = px.line_polar(
    df_melted,
    r='value',
    theta='metric',
    color='state',
    line_close=True,
    color_discrete_sequence=px.colors.qualitative.Vivid,
    template="plotly_white"
)

# --- Layout styling for Looker Studio integration ---
fig_radar.update_layout(
    polar=dict(
        bgcolor='rgba(0,0,0,0)',
        radialaxis=dict(visible=True, showticklabels=False, gridcolor='lightgrey', range=[0, 1]),
        angularaxis=dict(gridcolor='lightgrey', tickfont=dict(size=11))
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=80, r=80, t=20, b=60),
    autosize=True,
    legend=dict(
        orientation="h",
        y=-0.25,
        x=0.5,
        xanchor="center"
    )
)

# --- Output and Export to HTML ---
fig_radar.show()
fig_radar.write_html("customer_persona_radar_final.html", include_plotlyjs='cdn', full_html=False)